In [ ]:
import json
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from torch import optim
from torch.nn.functional import softmax
from model import *
from defaultParameter import*
from utils import *
from random import randint, shuffle
from train import getTrainSet

testSet, label2IndexRumor, label2IndexStance = getTrainSet('../dataset/semeval2017-task8/')

device = torch.device('cpu')
model = MTUS(numRumorClass=len(label2IndexRumor), numStanceClass=len(label2IndexStance))
model.load_state_dict(torch.load())
model = model.set_device(device)
loss_func = torch.nn.CrossEntropyLoss(reduction='sum').to(device)

In [ ]:
model.eval()
rumorTrue = []
stanceTrue = []
rumorPre = []
stancePre = []
totalLossRumor = 0.
totalLossStance = 0.
for i in range(len(testSet)):
    x = testSet[i][0].to(device)
    rumorTag = testSet[i][1].to(device)
    stanceTag = testSet[i][2].to(device)
    rumorTrue += testSet[i][1].tolist()
    stanceTrue += testSet[i][2].tolist()

    pRumor = model.forwardRumor(x)
    pStance = model.forwardStance(x).view(-1, len(label2IndexStance))
    loss = loss_func(pRumor, rumorTag)
    totalLossRumor += loss
    loss = loss_func(pStance, stanceTag)
    totalLossStance += loss
    pRumor = softmax(pRumor, 1)
    rumorPre += pRumor.max(dim=1)[1].tolist()
    pStance = softmax(pStance, 1)
    stancePre += pStance.max(dim=1)[1].tolist()

microF1Rumor = f1_score(rumorTrue, rumorPre, labels=[0,1,2], average='micro')
macroF1Rumor = f1_score(rumorTrue, rumorPre, labels=[0,1,2], average='macro')
microF1Stance = f1_score(stanceTrue, stancePre, labels=[0,1,2], average='micro')
macroF1Stance = f1_score(stanceTrue, stancePre, labels=[0,1,2], average='macro')
accuracyRumor = (np.array(rumorTrue) == np.array(rumorPre)).sum() / len(rumorPre)
accuracyStance = (np.array(stanceTrue) == np.array(stancePre)).sum() / len(stancePre)
print('rumor detection:')
print('average loss: {:f}, accuracy: {:f}, micro-f1: {:f}, macro-f1: {:f}'.format(
    totalLossRumor / len(testSet), accuracyRumor, microF1Rumor, macroF1Rumor
))
print('stance analyze:')
print('average loss: {:f}, accuracy: {:f}, micro-f1: {:f}, macro-f1: {:f}'.format(
    totalLossStance / len(testSet), accuracyStance, microF1Stance, macroF1Stance
))